# Structured Output

This example refers to [LangChain quickstart](https://python.langchain.com/docs/introduction/) and [LangChain開發手冊(旗標)](https://www.tenlong.com.tw/products/9789863127918)

In [24]:
!pip install langchain --quiet
!pip install langchain_openai --quiet
!pip install rich --quiet

In [25]:
import os
from google.colab import userdata

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [26]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

## Structred Output & Output Parsers

In [27]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
str_parser = StrOutputParser()

In [28]:
message = model.invoke("請提供一個國家的名稱和首都, 使用繁體中文")
print(message.content)

國家名稱：日本  
首都：東京


In [29]:
str_parser.invoke(message)

'國家名稱：日本  \n首都：東京'

### JSON Parser

In [30]:
json_parser = JsonOutputParser()
format_instructions = json_parser.get_format_instructions()
print(format_instructions)

Return a JSON object.


In [31]:
message = model.invoke("請提供一個國家的名稱和首都,"
                    f"{format_instructions}, 使用台灣語言")
print(message.content)

{
  "國家": "日本",
  "首都": "東京"
}


In [32]:
json_output = json_parser.invoke(message)
print(json_output)

{'國家': '日本', '首都': '東京'}


### CSV Parser

In [33]:
from langchain_core.output_parsers import (
    CommaSeparatedListOutputParser)

In [34]:
list_parser = CommaSeparatedListOutputParser()
print(list_parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [35]:
# 建立提示模板
prompt = PromptTemplate.from_template(
    "請說出國家{city}的知名景點\n{instructions}"
).partial(instructions=list_parser.get_format_instructions())
response = model.invoke(prompt.format(city='台灣'))
print(response.content)

台北101, 故宮博物院, 九份, 日月潭, 阿里山, 太魯閣國家公園, 龍騰斷橋, 澎湖列島, 垦丁國家公園, 士林夜市


In [36]:
pprint(list_parser.invoke(response))

[
    '台北101',
    '故宮博物院',
    '九份',
    '日月潭',
    '阿里山',
    '太魯閣國家公園',
    '龍騰斷橋',
    '澎湖列島',
    '垦丁國家公園',
    '士林夜市'
]

### 使用Pydanic自訂class

#### 旅遊計劃書

In [37]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, model_validator
from typing import List
from rich import print as pprint

In [38]:
class TravelPlan(BaseModel):
    destination: str = Field(description="旅遊目的地, 如日本北海道")
    activities: List[str] = Field(description="推薦的活動")
    budget: float = Field(description="預算範圍,單位新台幣")
    accommodation: List[str] = Field(description="住宿選項")

In [39]:
parser = PydanticOutputParser(pydantic_object=TravelPlan)
format_instructions = parser.get_format_instructions()
pprint(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": 
"array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": 
["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"destination": {"description": "旅遊目的地, 如日本北海道", "title": "Destination", "type": 
"string"}, "activities": {"description": "推薦的活動", "items": {"type": "string"}, "title": "Activities", "type": 
"array"}, "budget": {"description": "預算範圍,單位新台幣", "title": "Budget", "type": "number"}, "accommodation": 
{"description": "住宿選項", "items": {"type": "string"}, "title": "Accommodation", "type": "array"}}, "required": 
["destination", "activities", "budget", "accommodation"]}
```

In [46]:
prompt = ChatPromptTemplate.from_messages(
    [("system","使用繁體中文並根據使用者要求推薦出適合的旅遊計劃,\n"
               "{format_instructions}"),
     ("human","{query}")
    ]
)
new_prompt = prompt.partial(format_instructions=format_instructions)

In [41]:
user_query = "我喜歡潛水以及在日落時散步, 所以想要安排一個海邊假期"
user_prompt = new_prompt.invoke({"query": user_query})
response = model.invoke(user_prompt)
pprint(response.content)

{
  "destination": "菲律賓薄荷島",
  "activities": [
    "潛水探索珊瑚礁",
    "夜晚在海灘散步觀賞日落",
    "浮潛觀賞海洋生物",
    "參加島嶼 hopping 之旅",
    "享用當地美食海鮮"
  ],
  "budget": 30000,
  "accommodation": [
    "海景飯店",
    "度假村",
    "民宿"
  ]
}

In [42]:
parser_output = parser.invoke(response)
pprint(parser_output)

TravelPlan(
    destination='菲律賓薄荷島',
    activities=[
        '潛水探索珊瑚礁',
        '夜晚在海灘散步觀賞日落',
        '浮潛觀賞海洋生物',
        '參加島嶼 hopping 之旅',
        '享用當地美食海鮮'
    ],
    budget=30000.0,
    accommodation=['海景飯店', '度假村', '民宿']
)

#### Use @model_validator to check Output format

In [51]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [52]:
# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke about cats"})
parser.invoke(output)

Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!')

In [58]:
output = prompt_and_model.invoke({"query": "寫一份飲料店創業計畫書"})

In [59]:
parser.invoke(output)

OutputParserException: Failed to parse Joke from completion {"setup": "\u5982\u4f55\u5728\u98f2\u6599\u5e02\u5834\u6210\u529f\u5275\u696d\uff1f", "punchline": "\u63d0\u4f9b\u5275\u65b0\u7684\u98f2\u54c1\u548c\u5353\u8d8a\u7684\u9867\u5ba2\u670d\u52d9\uff01"}. Got: 1 validation error for Joke
  Value error, Badly formed question! [type=value_error, input_value={'setup': '如何在飲...越的顧客服務！'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 